Julia code for Kitaev model.
\section{Kitaev model without symmetry}

In [1]:
using SparseArrays
using Arpack
using BenchmarkTools

function index(N1::Int, N2::Int, c::Int, r::Int, atom::Int)
    #Calculate the index of the atoms in the (r,c) unit cell
    #A-sublattice atom = 0; B-subkattice: atom = 1
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c
    ind = 2 * n + 1 + atom
    return ind
    end;

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
    end;
    
function StateConfig(tag::Int, N1::Int, N2::Int)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
    end;

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
    end;

function KitaevRhom(J::Array{Float64,1}, N1::Int, N2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice
    Inputes: N1, N2: column and row numbers
             J: coupling constants
    Outpus: The Non-zero Hamiltonian element H[i] is in the 
            position (row[i], col[i])
            H, row, col : type: list =#
    SiteNum = N1 * N2 * 2
    dim = 2^SiteNum
    
    Ham = spzeros(Float64, dim, dim)

    for tag = 0: dim-1

        for r = 0: N2-1
            for c = 0: N1-1
                ind = index(N1, N2, c, r, 0)
                
                # Sx-Sx
                next = index(N1, N2, c, r, 1)
                ntag = flip(tag, ind, next, N1, N2)
                Ham[ntag+1, tag+1] += -J[1] /4
                
                # Sy-Sy
                next = index(N1, N2, c - 1, r, 1) 
                ntag = flip(tag, ind, next, N1, N2) 
                Ham[ntag+1, tag+1] += J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                        
                #Sz-Sz
                next = index(N1, N2, c, r - 1, 1)
                Ham[tag+1, tag+1] += - J[3] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
            end
        end                       
    end
    return Ham
    end;

In [2]:
N1 = 2; N2 = 3
J = -[1.0,1.0,1.0]

@benchmark H = KitaevRhom(J, N1, N2)    

BenchmarkTools.Trial: 
  memory estimate:  2.03 MiB
  allocs estimate:  35
  --------------
  minimum time:     15.407 ms (0.00% GC)
  median time:      16.293 ms (0.00% GC)
  mean time:        16.326 ms (0.00% GC)
  maximum time:     19.881 ms (0.00% GC)
  --------------
  samples:          301
  evals/sample:     1

\section{Kitaev model with translation symmetry}
\subsection{Momentum basis}
add notes!

In [2]:
# Tool functions
using SparseArrays
using Arpack
using BenchmarkTools

function index(N1::Int, N2::Int, c::Int, r::Int, atom::Int)
    #Calculate the index of the atoms in the (r,c) unit cell
    #A-sublattice atom = 0; B-subkattice: atom = 1
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c
    ind = 2 * n + 1 + atom
    return ind
    end;

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
    end;

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
    end;

function cyclebits(tag::Int, n1::Int, n2::Int, N1::Int, N2::Int)
    #=Performs a cyclic permutations of n1 steps to the right and n2 steps upward.
       Inputs: tag: tag of the reference state
               n1: cycle times of n1-direction, columns,if n1=0, no cycle in columns
               n2: cycle times of n2-direction, rows,if n2=0, no cycle in rows
               N1: number of columns
               N2: number of rows
       Outputs: ntag: the tag of the outcoming state=#
    SiteNum = N1 * N2 * 2
    ntag = 0
    for r = 0: N2-1
        for c = 0: N1-1
            for atom = 0: 1
                ind = index(N1, N2, c, r, atom)
                # notice that c,r should be positive in julia, -1%3 = -1 instead of 2
                nind = index(N1, N2, c + n1, r + n2, atom) # n1: ->right, n2 -> up
                ntag += 2^(SiteNum - nind) * bits(tag, ind, N1, N2)
            end
        end
    end
    return ntag
    end;

#=
function findindex(tag::Int, mb::Array{Int,1})
    # find the index of a tag in mb
    # binary search
    dim = length(mb)
    low = 1
    high = dim
    while low <= high
        mid = low + div(high - low, 2)
        if tag < mb[mid]
            high = mid -1
        elseif tag > mb[mid]
            low = mid + 1
        elseif tag == mb[mid]
            return mid
        end
    end
    if low > high
        return -1
    end
    end;
=#

function findindex(tag::Int, mb::Array{Int,1})
    # find the index of tag in a sorted mb
    dim = length(mb)
    ind = searchsortedfirst(mb, tag)
    if ind <= dim
        if mb[ind] == tag
            return ind
        end
    else
        return -ind
    end
    return -ind
end

function findposition(tag::Int, mb::Array{Int,1}, first::Int, last::Int)
    low = first
    high = last
    while low <= high
        mid = low + div(high - low, 2)
        if tag < mb[mid]
            high = mid -1
        elseif tag > mb[mid]
            low = mid + 1
        elseif tag == mb[mid]
            return mid
        end
    end
    
    if low > high
        return -low
    end
end

findposition (generic function with 1 method)

In [9]:
a = [1,2,3,5,6]
findposition(3, a,1,3)

3

In [13]:
"""Momentum basis generation"""
struct SymmetricBasis
    basis::Array{Int,1}
    norm::Array{Float64,1}
    ref_state::Array{Int,1}
    phase_real::Array{Float64,1}
    phase_img::Array{Float64,1}
end

function MomBasis(N1::Int, N2::Int, k1::Int, k2::Int)
    #=Momentum basis list of a given k1, k2
       Inputs: N1: number of columns
               N2: number of rows    
               k1: momentum in n1-direction
               k2: momentum in n2-direction

       Outpus: mb: momentum basis
               ref: reference states
               trans: translation operation to corresponding reference state
               factor: renormalization factor 
               =#
    
    dim = 2^(N1 * N2 * 2)
    mb = [x for x = 1: dim] # momentum basis
    ref = zeros(Int, dim)
    trans = zeros(Int, dim)
    factor = -ones(dim)
    
    error = 10^(-10)
    
    k1 = -2im * pi/N1 * k1
    k2 = -2im * pi/N2 * k2
    
    for tag = 0: (dim - 1)
        count = 0 # count number of non-equicalent translation operations
        fac = 0.0 + 0.0im # accumulate renormalization factor
        if factor[tag + 1] < 0
            for n1 = 0: N1-1, n2 = 0: N2-1
                ntag = cyclebits(tag, n1, n2, N1, N2)
                ref[ntag + 1] = tag
                # note the first translation operation that obtains this state
                if factor[ntag + 1] < 0
                    trans[ntag + 1] = (N1 - n1)%N1 *10 + (N2 - n2)%N2
                end
                
                if ntag > tag
                    factor[ntag + 1] = 0
                elseif ntag == tag
                    fac += exp(k1*n1 + k2*n2)
                    count += 1
                    factor[ntag + 1] = 1
                end
            end
            if abs(fac) < error
                fac = 0
            end
            factor[tag + 1] *= abs(fac)* sqrt(N1*N2/count)
        end
    end
    
    for i = 1: dim
        mb[i] = mb[i] * sign(factor[i])
    end 
    
    factor = filter!(x->x != 0, factor)
    mb = filter!(x->x != 0, mb)
    mb = replace!(x -> x-1, mb)
    return mb, factor, trans, ref                     
    end

function MomBasis_opt(N1::Int, N2::Int, k1::Int, k2::Int)
    #=Momentum basis list of a given k1, k2
       Inputs: N1: number of columns
               N2: number of rows    
               k1: momentum in n1-direction
               k2: momentum in n2-direction

       Outpus: mb: momentum basis
               ref: reference states
               phase: translation operation to corresponding reference state
               norm: renormalization factor 
               =#
    
    dim = 2^(N1 * N2 * 2)
    mb = [x for x = 1: dim] # momentum basis
    ref, norm = zeros(Int, dim), -ones(dim)
    ph_real, ph_img = zeros(Float64, dim), zeros(Float64, dim)
    
    error = 10^(-10)
    
    k1, k2 = 2* pi/N1 * k1, 2* pi/N2 * k2
    
    for tag = 0: (dim - 1)
        count = 0 # count number of non-equicalent translation operations
        fac = 0.0 + 0.0im # accumulate renormalization factor
        if norm[tag + 1] < 0
            for n1 = 0: N1-1, n2 = 0: N2-1
                ntag = cyclebits(tag, n1, n2, N1, N2)
                ref[ntag + 1] = tag
                # note the first translation operation that obtains this state
                if norm[ntag + 1] < 0
                    ph_real[ntag + 1] = cos(n1* k1 + n2* k2)
                    ph_img[ntag + 1] = cos(n1* k1 + n2* k2)
                end
                
                if ntag > tag
                    norm[ntag + 1] = 0
                elseif ntag == tag
                    fac += exp(k1*n1 + k2*n2)
                    count += 1
                    norm[ntag + 1] = 1
                end
            end
            if abs(fac) < error
                fac = 0
            end
            norm[tag + 1] *= abs(fac)* sqrt(N1*N2/count)
        end
    end
    
    for i = 1: dim
        mb[i] = mb[i] * sign(norm[i])
    end 
    
    factor = filter!(x->x != 0, norm)
    mb = filter!(x->x != 0, mb)
    mb = replace!(x -> x-1, mb)
    return SymmetricBasis(mb, norm, ref, ph_real, ph_img)                     
    end

MomBasis_opt (generic function with 1 method)

In [6]:
N1 = 3; N2 = 4
k1 = 0; k2 = 0
mb = MomBasis_opt(N1, N2, k1, k2);

In [103]:
N1 = 3; N2 = 2
k1 = 0; k2 = 0
@benchmark MomBasis(N1, N2, k1, k2)

BenchmarkTools.Trial: 
  memory estimate:  160.44 KiB
  allocs estimate:  11
  --------------
  minimum time:     825.945 μs (0.00% GC)
  median time:      856.458 μs (0.00% GC)
  mean time:        912.239 μs (1.56% GC)
  maximum time:     5.234 ms (76.10% GC)
  --------------
  samples:          5464
  evals/sample:     1

\subsection{Hamiltonian matrix}
add notes

In [69]:
# Hamiltonian matrix
using SparseArrays
                           
function KitaevH(J::Array{Float64,1}, N1::Int, N2::Int, k1::Int, k2::Int, mb::Array{Int,1}
                 , factor::Array{Float64,1}, trans::Array{Int,1}, ref::Array{Int,1})
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice for a given k (with translation symmetry)
       N1: number of unit cells in n1-direction, column number
       N2: number of unit cells in n1-direction, row number
    
       Inputes: 
           N1, N2: width and length of the rhomboid cluster
           k1, k2: momentund
        
       Outputs: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        =#
    dim = length(mb)
    f1 = - 2 * pi / N1
    f2 = - 2 * pi / N2
    
    H_real = spzeros(Float64, dim, dim)
    H_img = spzeros(Float64, dim, dim)
    
    for tag in mb
        i = findindex(tag, mb)
        for r = 0: N2-1
            for c = 0: N1-1
                si = index(N1, N2, c, r, 0) # start index
                
                # Sz-Sz
                ni = index(N1, N2, c, r - 1, 1)
                H_real[i, i] += - J[3] * (bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)
                
                # Sx-Sx
                ni = index(N1, N2, c, r, 1) # next index
                ntag = flip(tag, si, ni, N1, N2)
                rtag = ref[ntag + 1]
                j = findindex(rtag, mb)
                if j > 0
                    l1 = div(trans[ntag+1], 10)
                    l2 = trans[ntag + 1]%10
                    
                    phase_real = cos(f1*k1*l1 + f2*k2*l2) * factor[j]/ factor[i]
                    phase_img = sin(f1*k1*l1 + f2*k2*l2) * factor[j]/ factor[i]
                    H_real[j, i] += -J[1]/4 * phase_real
                    H_img[j, i] += -J[1]/4 * phase_img
                end
             
                # Sy-Sy
                ni = index(N1, N2, c - 1, r, 1)
                ntag = flip(tag, si, ni, N1, N2)
                rtag = ref[ntag + 1]
                j = findindex(rtag, mb)
                if j > 0
                    l1 = div(trans[ntag+1], 10)
                    l2 = trans[ntag + 1]%10
                    
                    phase_real = cos(f1*k1*l1 + f2*k2*l2) * factor[j]/ factor[i]
                    phase_img = sin(f1*k1*l1 + f2*k2*l2) * factor[j]/ factor[i]
                    H_real[j, i] += J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5) * phase_real
                    H_img[j, i] += J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5) * phase_img
                end
            end
        end
    end

    H_real = (H_real + transpose(H_real))/2
    H_img = (H_img - transpose(H_img))/2
    
    H = H_real + 1im * H_img
    #H = (H + transpose(conj(H)))/2
    return H
    end;

function KitaevH_opt(J::Array{Float64,1}, N1::Int, N2::Int, k1::Int, k2::Int, mb::Array{Int,1}
                 , factor::Array{Float64,1}, trans::Array{Int,1}, ref::Array{Int,1})
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice for a given k (with translation symmetry)
       N1: number of unit cells in n1-direction, column number
       N2: number of unit cells in n1-direction, row number
    
       Inputes: 
           N1, N2: width and length of the rhomboid cluster
           k1, k2: momentund
        
       Outputs: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        =#
    dim = length(mb)
    max_num = dim * (N1* N2* 2 + 1)
    
    k1, k2 = -2* pi* k1/ N1, -2* pi* k2/ N2

    #H_real = spzeros(Float64, dim, dim)
    #H_img = spzeros(Float64, dim, dim)
    
    col, row = zeros(Int, max_num), (dim+1)* ones(Int, max_num)
    data_real, data_img = zeros(max_num), zeros(max_num)
    
    first, last = 1, 1 
    
    for tag in mb
        i = findindex(tag, mb)
        for r = 0: N2-1
            for c = 0: N1-1
                si = index(N1, N2, c, r, 0) # start index
                
                """Sz-Sz"""
                ni = index(N1, N2, c, r - 1, 1)
                val_real = -J[3]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)
                
                # find position
                pos = findposition(i, row, first, last)
                if pos > 0
                    data_real[pos] += val_real
                else
                    col[last] = i
                    pos = -pos
                    if pos == last
                        row[pos] = i
                        data_real[pos] = val_real
                    else
                        for p = (last-1): -1: pos
                            row[p+1] = row[p]
                            data_real[p+1] = data_real[p]
                        end
                        row[pos] = i
                        data_real[pos] = val_real
                    end
                    last += 1
                end
                
    
                """Sx-Sx"""
                ni = index(N1, N2, c, r, 1) # next index
                ntag = flip(tag, si, ni, N1, N2)
                rtag = ref[ntag + 1]
                j = findindex(rtag, mb)
                if j > 0
                    l1 = div(trans[ntag+1], 10)
                    l2 = trans[ntag + 1]%10
                    
                    phase_real = cos(k1*l1 + k2*l2) * factor[j]/ factor[i]
                    phase_img = sin(k1*l1 + k2*l2) * factor[j]/ factor[i]
                    val_real = -J[1]/4 * phase_real
                    val_img = -J[1]/4 * phase_img

                    # find position
                    pos = findposition(j, row, first, last)
                    if pos > 0
                        data_real[pos] += val_real
                        data_img[pos] += val_img
                    else
                        col[last] = i
                        pos = -pos
                        if pos == last
                            row[pos] = j
                            data_real[pos] = val_real
                            data_img[pos] = val_img
                        else
                            for p = (last-1): -1: pos
                                row[p+1] = row[p]
                                data_real[p+1] = data_real[p]
                                data_img[p+1] = data_img[p]
                            end
                            row[pos] = j
                            data_real[pos] = val_real
                            data_img[pos] = val_img
                        end
                        last += 1
                    end
                    
                end
             

                """Sy-Sy"""
                ni = index(N1, N2, c - 1, r, 1)
                ntag = flip(tag, si, ni, N1, N2)
                rtag = ref[ntag + 1]
                j = findindex(rtag, mb)
                if j > 0
                    l1 = div(trans[ntag+1], 10)
                    l2 = trans[ntag + 1]%10
                    
                    phase_real = cos(k1*l1 + k2*l2) * factor[j]/ factor[i]
                    phase_img = sin(k1*l1 + k2*l2) * factor[j]/ factor[i]
                    val_real = J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)* phase_real
                    val_img = J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)* phase_img
                   
                    # find position
                    pos = findposition(j, row, first, last)
                    if pos > 0
                        data_real[pos] += val_real
                        data_img[pos] += val_img
                    else
                        col[last] = i
                        pos = -pos
                        if pos == last
                            row[pos] = j
                            data_real[pos] = val_real
                            data_img[pos] = val_img
                        else
                            for p = (last-1): -1: pos
                                row[p+1] = row[p]
                                data_real[p+1] = data_real[p]
                                data_img[p+1] = data_img[p]
                            end
                            row[pos] = j
                            data_real[pos] = val_real
                            data_img[pos] = val_img
                        end
                        last += 1
                    end                   
                end
            end
        end
        first = last
    end
    last -= 1
    data = data_real[1:last] + 1im * data_img[1:last]
    H = sparse(row[1:last], col[1:last], data, dim, dim)
    return H
end

KitaevH_opt (generic function with 1 method)

In [82]:
"""benchmark for new Hamiltonian matrix"""
N1 = 3; N2 = 3;
k1 = 0; k2 = 0;
J = -[1.0,1.0,1.0]
mb, factor, trans, ref = MomBasis(N1, N2, k1, k2);
#row, col, data = KitaevH_opt(J, N1, N2, k1, k2, mb, factor, trans, ref)

#println(col)
#println(row)
#println(data)

H_opt = KitaevH_opt(J, N1, N2, k1, k2, mb, factor, trans, ref)
e_opt,x = eigs(H_opt, which = :SR)
println(real(e_opt/(N1*N2*2)))

[-0.198493, -0.195481, -0.195481, -0.195481, -0.194736, -0.194736]


In [79]:
"""benchmark for old Hamiltonian matrix"""
N1 = 3; N2 = 3;
k1 = 0; k2 = 0;
J = -[1.0,1.0,1.0]
mb, factor, trans, ref = MomBasis(N1, N2, k1, k2);
@benchmark H = KitaevH(J, N1, N2, k1, k2, mb, factor, trans, ref)
#e,x = eigs(H, which = :SR)
#println(real(e/(N1*N2*2)))

BenchmarkTools.Trial: 
  memory estimate:  66.60 MiB
  allocs estimate:  104
  --------------
  minimum time:     1.315 s (0.00% GC)
  median time:      1.377 s (2.66% GC)
  mean time:        1.370 s (3.15% GC)
  maximum time:     1.412 s (0.80% GC)
  --------------
  samples:          4
  evals/sample:     1